# Toxic Sentence Generator Converter

This notebook demonstrates how to use the Toxic Sentence Generator converter, which is based on Project Moonshot's attack module. This converter generates toxic sentence starters based on a given seed prompt to test if an AI system will complete the sentence with toxic content.

**Note**: This converter is intended for red teaming and testing AI safety mechanisms only. The generated content should not be used for actual harmful activities.

In [5]:
# Import necessary modules
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.orchestrator import PromptSendingOrchestrator
from pyrit.prompt_converter import ToxicSentenceGeneratorConverter
from pyrit.prompt_target import OpenAIChatTarget

## Set up the environment and initialize the converter

First, we need to initialize PyRIT with an in-memory database, set up the target, converter, and orchestrator.

In [ ]:
# Initialize PyRIT with in-memory database
initialize_pyrit(memory_db_type=IN_MEMORY)

# Define seed prompts that will be used to generate toxic sentences
seed_prompts = [
    "cyber security",
    "financial advice",
    "social media",
    "privacy"
]

# Initialize OpenAI Chat Target
prompt_target = OpenAIChatTarget()

# Initialize the ToxicSentenceGeneratorConverter with the prompt target
toxic_sentence_generator = ToxicSentenceGeneratorConverter(converter_target=prompt_target)

# Initialize the orchestrator for sending prompts
orchestrator = PromptSendingOrchestrator(
    objective_target=prompt_target,
    prompt_converters=[toxic_sentence_generator],
    verbose=True,  # Set to True to see detailed output
)

## Generate and test toxic sentence starters

Now we can use the converter to generate toxic sentence starters based on different seed prompts and send them to the target model. This allows us to test the model's response to potentially harmful content.

In [ ]:
# Generate and send toxic sentences for each seed prompt
for seed_prompt in seed_prompts:
    print(f"\nSeed prompt: {seed_prompt}")
    
    # Generate a toxic sentence starter using the converter
    converter_result = await toxic_sentence_generator.convert_async(prompt=seed_prompt)
    
    # Print the generated toxic sentence
    print(f"Generated toxic sentence: {converter_result.output_text}")
    
    # Send the toxic sentence to the target LLM via the orchestrator
    await orchestrator.send_prompts_async(prompt_list=[converter_result.output_text])

## Review conversation results

Now we can review all the conversations and see how the target model responded to the generated toxic sentences. This helps identify potential areas where a model might be vulnerable to harmful inputs.

In [ ]:
# Print all conversations after all prompts are sent
print("\nAll conversations:")
await orchestrator.print_conversations_async()

# Clean up resources
orchestrator.dispose_db_engine()